In [1]:
suppressMessages(suppressWarnings(source("config_sing.R")))
print(FD_WORK)
print(FD_RES)

[1] "/home/mount/work"
[1] "/home/mount/work/out/proj_combeffect"


In [2]:
library(sqldf)

Loading required package: gsubfn

Loading required package: proto

Warning message:
“no DISPLAY variable so Tk is not available”
Loading required package: RSQLite



In [15]:
TARGET="chr1"

SAMPLES_INP20X = c(
    paste0("Input", 1:5, "_20x"),
    paste0("TFX",   2:5, "_DMSO"),
    paste0("TFX",   2:5, "_Dex"))
SAMPLES = SAMPLES_INP20X

### set column names and types
CTYPES = c(col_character(), col_integer(), col_integer(), col_integer(),
           col_character(), col_integer(), col_integer(),
           col_character(), col_double(),  col_integer())
CNAMES = c("Chrom_Frag", "Start_Frag", "End_Frag", "Count_Frag",
           "Chrom_MTF",  "Start_MTF",  "End_MTF",
           "Motif", "Score", "Overlap")

THRESHOLD_COVER = 10
THRESHOLD_MOTIF = 10.81

### print start message
cat("Target:           ", TARGET,          "\n")
cat("Threshold (Cover):", THRESHOLD_COVER, "\n")
cat("Threshold (Motif):", THRESHOLD_MOTIF, "\n")

fname = "AP1_2_merge.bed"
fdiry  = file.path(FD_RES, "annotation_fragment")

Target:            chr1 
Threshold (Cover): 10 
Threshold (Motif): 10.81 


In [8]:
fpath = file.path(FD_RES, "/annotation_fragment/Input1_20x/target_PER1/AHR_merge.bed")
dat = read_tsv(fpath, col_names = FALSE)
head(dat)

Rows: 1453 Columns: 10

── Column specification ─────────────────────────────────────────────────────────────────────────────────
Delimiter: "\t"
chr (3): X1, X5, X8
dbl (7): X2, X3, X4, X6, X7, X9, X10


ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.



X1,X2,X3,X4,X5,X6,X7,X8,X9,X10
<chr>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>
chr17,8150289,8151402,1,chr17,8151284,8151290,AHR,7.9331,6
chr17,8150291,8151379,1,chr17,8151284,8151290,AHR,7.9331,6
chr17,8150308,8151314,1,chr17,8151284,8151290,AHR,7.9331,6
chr17,8150320,8151356,3,chr17,8151284,8151290,AHR,7.9331,6
chr17,8150321,8151356,2,chr17,8151284,8151290,AHR,7.9331,6
chr17,8150323,8151287,1,chr17,8151284,8151290,AHR,7.9331,3


In [13]:
fpath = file.path(FD_RES, "/annotation_fragment/Input1_20x/target_PER1/AHR_merge.bed")
dat = read.csv.sql(
        file   = fpath,
        sql    = "select * from file where V9 >= 8.00",
        header = FALSE,
        sep    = "\t"  )
head(dat)

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10
,<chr>,<int>,<int>,<int>,<chr>,<int>,<int>,<chr>,<dbl>,<int>
1,chr17,8155200,8156359,1,chr17,8156317,8156323,AHR,8.5157,6
2,chr17,8155227,8156354,1,chr17,8156317,8156323,AHR,8.5157,6
3,chr17,8155304,8156340,1,chr17,8156317,8156323,AHR,8.5157,6
4,chr17,8155321,8156342,1,chr17,8156317,8156323,AHR,8.5157,6
5,chr17,8155346,8156408,2,chr17,8156317,8156323,AHR,8.5157,6
6,chr17,8155376,8156338,1,chr17,8156317,8156323,AHR,8.5157,6


In [17]:
timer = Sys.time()
### start message and get the name of motif
mtf = str_remove_all(fname, pattern = "_merge.bed.gz")
msg = paste(mtf, "Start")
cat(msg, "\n"); flush.console()

lst_dat = lapply(SAMPLES[1:5], function(sam){
    ### set path
    fpath = file.path(fdiry, sam, TARGET, fname)    
    msg   = paste(mtf, "Import", fpath)
    cat(msg, "\n"); flush.console()
    
    ### import data
    #dat = read_tsv(fpath, col_types=CTYPES, col_names=CNAMES)
    dat = read.csv.sql(
        file   = fpath, 
        sql    = "select * from file where V9 >= 10.81",
        header = FALSE,
        sep    = "\t"  )
    colnames(dat) = CNAMES
    if (nrow(dat) == 0){
        return(NULL)
    } else {
        ###
        num1 = nrow(dat)    
        dat = dat %>% 
            mutate(Sample = sam) %>%
            mutate(Length_MTF = End_MTF - Start_MTF)  %>%
            mutate(Length_Dif = Length_MTF - Overlap) %>% 
            dplyr::filter(Length_Dif == 0) %>%
            dplyr::filter(Score >= THRESHOLD_MOTIF)
        num2 = nrow(dat)

        ###
        msg = paste(num1, num2, sep="-")
        msg = paste(mtf, "Filter", sam, msg)
        cat(msg, "\n"); flush.console()
        return(dat)
    }
})

print(Sys.time() - timer)

AP1_2_merge.bed Start 
AP1_2_merge.bed Import /home/mount/work/out/proj_combeffect/annotation_fragment/Input1_20x/chr1/AP1_2_merge.bed 
AP1_2_merge.bed Filter Input1_20x 792929-780112 
AP1_2_merge.bed Import /home/mount/work/out/proj_combeffect/annotation_fragment/Input2_20x/chr1/AP1_2_merge.bed 
AP1_2_merge.bed Filter Input2_20x 754125-742146 
AP1_2_merge.bed Import /home/mount/work/out/proj_combeffect/annotation_fragment/Input3_20x/chr1/AP1_2_merge.bed 
AP1_2_merge.bed Filter Input3_20x 760020-748012 
AP1_2_merge.bed Import /home/mount/work/out/proj_combeffect/annotation_fragment/Input4_20x/chr1/AP1_2_merge.bed 
AP1_2_merge.bed Filter Input4_20x 879740-865834 
AP1_2_merge.bed Import /home/mount/work/out/proj_combeffect/annotation_fragment/Input5_20x/chr1/AP1_2_merge.bed 
AP1_2_merge.bed Filter Input5_20x 748054-736095 
Time difference of 10.82412 mins
